# Projeto Automação Web - Busca de Preços

### Objetivo:  Usar automações Web com selenium para buscar informações de produtos 


### Comentários:

- Comparação de fornecedores para insumos/produtos de interesse;

- Busca constante em sites para verificação de produtos disponíveis, promoções e valores;

- Objetivo: Se o valor dos produtos for abaixo de um preço limite definido, os produtos serão atualizados em uma planilha contendo a descrição, preço e link da oferta.

- Nesse caso, serão desenvolvidos códigos com Google Shopping e Buscapé, mas a estrutura lógica definida para as funções persiste para qualquer site. 

### Outra opção:

- APIs

### Informações disponíveis

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade") e os termos que vamos querer evitar nas nossas buscas.

### Etapas do projeto:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos;
- O mesmo para o Buscapé;
- Exportar a tabela compilada com os arquivos para formato excel.

## Importação das bibliotecas principais para realização do projeto

### Instalação da biblioteca _Selenium_

- A biblioteca selenium é uma das principais ferramentas para recolhimentos de informações na internet (**_Web Scrapping_**). Caso não tenha instalado no seu computador pessoal, é necessário executar o comando "pip install --upgrade selenium" dentro do prompt do anaconda;
- Importante verificar a versão do navegador (aconselhável Google Chrome), e acessar a página https://chromedriver.chromium.org/downloads para baixar a versão compatível com o navegador; 
- Após fazer o download do arquivo contendo a versão do _chromedriver_ compatível com seu computador pessoal, é necessário descompactar o arquivo .zip e colar na pasta pessoal do anaconda3. 

In [20]:
# Importação das bibliotecas para criação do navegador
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

# Biblioteca para manipulação dos dados
import pandas as pd

# Modificando o diretório do trabalho
import os 
path = r'C:\Users\cador\OneDrive\Área de Trabalho\Projetos Github\Python - Web Scrapping\WS - Pesquisa de precos'
os.chdir(path)

# criar o navegador
nav = webdriver.Chrome()

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel("buscas.xlsx")
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


### Definição da função de busca dentro do _Google Shopping_ - Etapas

- Entrada no navegador;
- Tratamento das variáveis contidas na tabela;
- Pesquisa do nome na aba _Google Shopping_;
- Para cada resultado pesquisado, verificação das condições limitadas pela tabela;
- Coleta e armazenamento dos dados com nome, preço e link da oferta.

In [16]:
def busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Entrada no navegador
    nav.get("https://www.google.com/")
    
    ## Tratamento dos valores que vieram da tabela
    # Colocando todos os termos em letras minúsculas - função variável(str).lower()
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    
    # Separação dos termos contidos na lista inicial
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    # Garantindo importação como número decimal
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    
    # Pesquisa do nome do produto no google- Cópia do caminho XPATH 
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(produto)
    nav.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
    
    #  Aba shopping - Lista de elementos para classe da Aba Shopping
    elementos = nav.find_elements(By.CLASS_NAME, 'hdtb-mitem')
    
    # Garantindo que o código percorre todas os elementos dentro da class e encontre a aba "Shopping"
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    # Lista de resultados da busca no google shopping
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'sh-dgr__grid-result')
    
    # Para cada resultado, o código verifica as condições estabelecidas: 
    # 1 - Termos banidos (tem_termos_banidos)
    # 2 - Termos produto (todos_termos_produto)
    # 3 - Limites mínimo e máximo dos preços
    
    # Lista  vazia como a função como resposta final
    lista_ofertas = [] 
    
    for resultado in lista_resultados:
        nome = resultado.find_element(By.CLASS_NAME, 'Xjkr3b').text
        nome = nome.lower()

        # Verificacão do nome - Produto tem  algum termo banido
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True
        
        # Verificação do nome - Checagem para o produto contém os termos de produtos de busca
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False
        
        # Verificação do nome em relação aos termos banidos e termos contido dentro do nome do produto
        if not tem_termos_banidos and tem_todos_termos_produto: # verificando o nome (sem ter)
            try:
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                # Edição do número vindo da página web 
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                
                # Verificação dos limites dentro dos limites máximo e mínimo
                if preco_minimo <= preco <= preco_maximo:
                    elemento_link = resultado.find_element(By.CLASS_NAME, 'aULzUe')
                    elemento_pai = elemento_link.find_element(By.XPATH, '..')
                    link = elemento_pai.get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
            
    return lista_ofertas

### Definição da função de busca dentro do Buscapé - Etapas

- Entrada no navegador;
- Tratamento das variáveis contidas na tabela;
- Pesquisa do nome na aba _Google Shopping_;
- Para cada resultado pesquisado, verificação das condições limitadas pela tabela;
- Coleta e armazenamento dos dados com nome, preço e link da oferta.

In [17]:
def busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo):
    
    # Entrada no navegador
    nav.get("https://www.buscape.com.br/")
    
    ## Tratamento dos valores que vieram da tabela
    # Colocando todos os termos em letras minúsculas - função variável(str).lower()
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    
    # Separação dos termos contidos na lista inicial
    lista_termos_banidos = termos_banidos.split(" ")
    lista_termos_produto = produto.split(" ")
    
    # Garantindo importação como número decimal
    preco_maximo = float(preco_maximo)
    preco_minimo = float(preco_minimo)
    
    # Pesquisa pelo produto no Buscapé
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto)
    nav.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
   
    # Lista de resultados da busca do Buscapé
    # Adicionando uma pausa de 5 s para o carregamento dinâmico das opções dentro do site
    time.sleep(5)
    lista_resultados = nav.find_elements(By.CLASS_NAME, 'Cell_Content__1630r')
    
    # Lista  vazia como a função como resposta final
    lista_ofertas = []
    
    # Para cada resultado, o código verifica as condições estabelecidas: 
    # 1 - Termos banidos (tem_termos_banidos)
    # 2 - Termos produto (todos_termos_produto)
    # 3 - Limites mínimo e máximo dos preços
    
    for resultado in lista_resultados:
        try:
            preco = resultado.find_element(By.CLASS_NAME, 'CellPrice_MainValue__3s0iP').text
            nome = resultado.get_attribute('title')
            nome = nome.lower()
            link = resultado.get_attribute('href')
            
            # Verificacão do nome - Produto tem  algum termo banido
            tem_termos_banidos = False
            for palavra in lista_termos_banidos:
                if palavra in nome:
                    tem_termos_banidos = True  
                    
            # Verificação do nome - Checagem para o produto contém os termos de produtos de busca
            tem_todos_termos_produto = True
            for palavra in lista_termos_produto:
                if palavra not in nome:
                    tem_todos_termos_produto = False
                    
            # Verificação dos limites dentro dos limites máximo e mínimo
            if not tem_termos_banidos and tem_todos_termos_produto:
                preco = preco.replace("R$", "").replace(" ", "").replace(".", "").replace(",", ".")
                preco = float(preco)
                if preco_minimo <= preco <= preco_maximo:
                    lista_ofertas.append((nome, preco, link))
        except:
            pass
        
    return lista_ofertas

### Contrução da nossa lista de ofertas encontradas

- A construção da tabela é dinâmica de acordo com o sistema de buscas dos sites.

In [18]:
# Criando uma tabela vazia para armazenamento das ofertas
tabela_ofertas = pd.DataFrame()

# Percorrendo as linhas das tabela produtos contendo as características
for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]
    
    lista_ofertas_google_shopping = busca_google_shopping(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_google_shopping)
    else:
        tabela_google_shopping = None
        
    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link'])
        tabela_ofertas = tabela_ofertas.append(tabela_buscape)
    else:
        tabela_buscape = None

display(tabela_ofertas)

,produto,preco,link
0,"iphone 11 apple (64gb) branco, tela de 6,1, 4g...",3290.0,https://www.google.com/url?url=https://www.ame...
0,placa de vídeo msi rtx 3060ti twin fan oc edit...,4499.0,https://www.google.com/url?url=https://www.sho...
1,placa de vídeo evga nvidia geforce rtx 3060 ti...,4499.9,https://www.google.com/url?url=https://www.ame...


### Exportar a base de ofertas para Excel

- Compilação dos dados em uma única tabela com formato .xlsx

In [19]:
# exportar pro excel
tabela_ofertas = tabela_ofertas.reset_index(drop = True)
tabela_ofertas.to_excel("Ofertas.xlsx", index = False)

### Comentários

- O código pode ser adaptado para maior quantidade de produtos, faixa de preços e aperfeiçoamento do sistema de buscas;
- Outros sites de busca podem ser incluídos sem maior dificuldade apenas com a adaptação dos códigos anteriores e classes dos respectivos endereços eletrônicos incluídos;
- É importante destacar que esse tipo de código necessita revisão constante pois é possível que haja mudanças na estruturação das classes pertencentes dentro de cada site pesquisado. 